In [2]:
import pandas as pd
import re
import ast

In [3]:
llava_path = 'outputs/llava_answer.csv'
llava_df = pd.read_csv(llava_path)

llava_df.head(3)

,filename,question,options,answer,category,llava_answer
0,BNATURE_3167.jpg,ছবিটি কোন ধর্মাবলম্বীদের প্রার্থনা স্থান?,"['হিন্দু', 'মুসলিম', 'বৌদ্ধ', 'খ্রিস্টান']",মুসলিম,Arts & History,The image depicts a mosque with Persian scrip...
1,BORNON_3694.jpg,এই মহিলাটি কোন ঐতিহ্যবাহী পোশাক পরেছেন?,"['শাড়ি', 'কুর্তা', 'লেহেঙ্গা', 'সালোয়ার কামিজ']",শাড়ি,Arts & History,answer: 1
2,CHITTRON_3491.png,চিত্র প্রদর্শিত মাছটি বাংলাদেশের কি হিসেবে অভি...,"['জাতীয় মাছ', 'সাধারণ মাছ', 'ছোট মাছ', 'কোনটি...",জাতীয় মাছ,Arts & History,answer: 1


In [4]:
def extract_number(strings):
    for s in strings:
        match = re.search(r'\d+', s)  
        if match:
            return int(match.group())  
    return -1 

In [5]:
# pf = llava_df[['filename'] == 'BORNON_3864.jpg']
# str(llava_df[llava_df['filename'] == 'BORNON_3864.jpg']['llava_answer'])
llava_df['llava_answer'] = llava_df['llava_answer'].apply(extract_number)

In [6]:
llava_df.head(3)

,filename,question,options,answer,category,llava_answer
0,BNATURE_3167.jpg,ছবিটি কোন ধর্মাবলম্বীদের প্রার্থনা স্থান?,"['হিন্দু', 'মুসলিম', 'বৌদ্ধ', 'খ্রিস্টান']",মুসলিম,Arts & History,-1
1,BORNON_3694.jpg,এই মহিলাটি কোন ঐতিহ্যবাহী পোশাক পরেছেন?,"['শাড়ি', 'কুর্তা', 'লেহেঙ্গা', 'সালোয়ার কামিজ']",শাড়ি,Arts & History,1
2,CHITTRON_3491.png,চিত্র প্রদর্শিত মাছটি বাংলাদেশের কি হিসেবে অভি...,"['জাতীয় মাছ', 'সাধারণ মাছ', 'ছোট মাছ', 'কোনটি...",জাতীয় মাছ,Arts & History,1


In [7]:
# llava_df.to_csv(llava_path, index=False)

In [8]:
def substitute_answer(row):
    idx = row['llava_answer']
    options = ast.literal_eval(row['options'])
    if idx == -1:# or idx-1 >= len(options):
        return -1
    # print(options, idx)
    # print(options[int(idx)-1])
    return options[int(idx)-1]

In [9]:
llava_df['llava_answer_parsed'] = llava_df.apply(lambda row: substitute_answer(row), axis=1)

In [10]:
llava_df.head(3)

,filename,question,options,answer,category,llava_answer,llava_answer_parsed
0,BNATURE_3167.jpg,ছবিটি কোন ধর্মাবলম্বীদের প্রার্থনা স্থান?,"['হিন্দু', 'মুসলিম', 'বৌদ্ধ', 'খ্রিস্টান']",মুসলিম,Arts & History,-1,-1
1,BORNON_3694.jpg,এই মহিলাটি কোন ঐতিহ্যবাহী পোশাক পরেছেন?,"['শাড়ি', 'কুর্তা', 'লেহেঙ্গা', 'সালোয়ার কামিজ']",শাড়ি,Arts & History,1,শাড়ি
2,CHITTRON_3491.png,চিত্র প্রদর্শিত মাছটি বাংলাদেশের কি হিসেবে অভি...,"['জাতীয় মাছ', 'সাধারণ মাছ', 'ছোট মাছ', 'কোনটি...",জাতীয় মাছ,Arts & History,1,জাতীয় মাছ


In [11]:
llava_df['llava_answer_parsed'].value_counts()

llava_answer_parsed
-1                       73
ফুটবল                    13
ক্রিকেট                  10
ঢাকা                      5
মসজিদ                     5
                         ..
জর্দা পান                 1
বৌদ্ধ পূর্ণিমা            1
১৪ এপ্রিল                 1
রাস্তার ধারে বসে আছেন     1
চাল-ডাল                   1
Name: count, Length: 381, dtype: int64